In [1]:
import torch
import torch.nn as nn

import math
from functools import partial
import numpy as np
from io import StringIO
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter1d
import sys
import time
import cv2

sys.path.append('/home/oscar_palfelt/MSc_thesis/ompl/py-bindings')
from ompl import base as ob
from ompl import control as oc
from ompl import geometric as og

cuda = True
DEVICE = torch.device("cuda" if cuda else "cpu")

In [2]:
# neural network parameters
mb_size = 256 # mini batch dim
h_Q_dim = 512*2 # encoder dim
h_P_dim = 512*2 # decoder dim

c = 0 # unsure of this constant
lr = 1e-4 # learning rate

# problem dimenc_dimsions
nDrawnSamples = 1 # number of dependent samples to draw during smapling
dim = 4 # (x, y, xdot, ydot)
dataElements = nDrawnSamples * dim + 2 * dim # sample (4D), init (4D), goal (4D)

z_dim = 2 # latent dim
X_dim = dim*nDrawnSamples # samples dim
y_dim = dim # reconstruction of the original point (unsused?)
c_dim = dataElements - dim # dimension of conditioning variable

In [3]:
# read in data from .txt file, re-arrange to allow drawing multiple depedent samples

filename = '/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_distributions/motion_planning/data/pathDataDifficult_GoodHeading_combined'
data = np.genfromtxt(filename, delimiter=',', dtype='d')
_, uniqueind = np.unique(data[:,4:], return_index=True, axis=0)
data = data[uniqueind] # don't use duplicate scenarios

numEntries = data.shape[0]
print(numEntries)

103253


In [4]:
# generate conditioning variable
filename = '/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_distributions/motion_planning/data_generation/map_difficult.png'
mapImg = cv2.imread(filename, 0)
blurImg = cv2.blur(mapImg, ksize=(3,3))
occGrid = np.clip(mapImg, 0, 1)
inflatedGrid = np.floor(blurImg / 255)

assert occGrid.shape[0] == occGrid.shape[1]
gridSize = occGrid.shape[0]

#conditionsOcc = np.tile(occGrid.reshape(1, gridSize ** 2), reps=(numEntries,1))
#cs = np.concatenate((data[0:numEntries,dim:dataElements], conditionsOcc), axis=1) # init (4D), goal (4D), occ(30x30)

cs = data[0:numEntries,dim*nDrawnSamples:dataElements]

In [5]:
# define pytorch networks
# based on https://github.com/Jackson-Kang/Pytorch-VAE-tutorial/blob/master/.ipynb_checkpoints/01_Variational_AutoEncoder-checkpoint.ipynb

class Encoder(nn.Module):
    
    def __init__(self, input_dim=X_dim+c_dim, hidden_dim=h_Q_dim, latent_dim=z_dim):
        super(Encoder, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        self.z_mu = nn.Linear(hidden_dim, latent_dim)
        self.z_logvar = nn.Linear(hidden_dim, latent_dim)
        
    def forward(self, x):

        seq = self.network(x)

        return self.z_mu(seq), self.z_logvar(seq)


class Decoder(nn.Module):
    def __init__(self, latent_dim=z_dim+c_dim, hidden_dim=h_P_dim, output_dim=X_dim):
        super(Decoder, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        
    def forward(self, x):
        
        return self.network(x)


class NeuralNetwork(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(NeuralNetwork, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder
    
    def reparameterization(self, mean, logvar):
        epsilon = torch.randn_like(mean).to(DEVICE)        # sampling epsilon        
        z = mean + torch.exp(0.5 * logvar) * epsilon       # reparameterization trick
        return z

    def forward(self, x, c, encode=True):
        if encode:
            z_mu, z_logvar = self.Encoder(torch.cat((x, c), dim=1))
            z = self.reparameterization(z_mu, z_logvar)

            y = self.Decoder(torch.cat((z, c), dim=1))
            
            return y, z_mu, z_logvar
        else:
            z = x
            y = self.Decoder(torch.cat((z, c), dim=1))    

            return y

network = torch.load('/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_distributions/network_difficult.pt')

In [10]:
# define planning problem

def getThresholdPathLengthObj(si):
     obj = ob.PathLengthOptimizationObjective(si)
     obj.setCostThreshold(ob.Cost(4.0))
     return obj


class MyStateSampler(ob.StateSampler):
    def __init__(self, space):
         super(MyStateSampler, self).__init__(space)
         self.name_ = "my sampler"
         self.bounds = space.getBounds()

    def sampleUniform(self, state):
        if np.random.uniform() < p:
            rndState = y[np.random.randint(low=0, high=y.shape[0]-1), :]
            state.setYaw(float(int(rndState[3] > 0) * np.arccos(np.dot(rndState[2:4] / np.linalg.norm(rndState[2:4]), [1, 0]))))
        else:
            rndState = [np.random.uniform(low=self.bounds.low[0], high=self.bounds.high[0]),
                        np.random.uniform(low=self.bounds.low[1], high=self.bounds.high[1]),
                        np.random.uniform(low=-np.pi, high=np.pi)]
            state.setYaw(rndState[2])

        state.setX(float(rndState[0]))
        state.setY(float(rndState[1]))


def allocStateSampler(space):
    return MyStateSampler(space)


def isStateValid(spaceInformation, state):
    # perform collision checking or check if other constraints are
    # satisfied
    u = int(np.floor(state.getX() * occGrid.shape[1])) # right pointing image axis
    v = int(np.floor(occGrid.shape[0] * (1 - state.getY()))) # down pointing image axis

    if spaceInformation.satisfiesBounds(state):
        return inflatedGrid[v,u] > 0


def problemDef(useNN=False):
    # construct the state space we are planning in
    #space = ob.DubinsStateSpace(turningRadius=0.07)
    space = ob.SE2StateSpace()

    # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(2)
    bounds.setLow(0.001)
    bounds.setHigh(0.999)
    space.setBounds(bounds)

    # set state sampler
    if useNN:
        space.setStateSamplerAllocator(ob.StateSamplerAllocator(allocStateSampler))

    # define a simple setup class
    ss = og.SimpleSetup(space)
    ss.setStateValidityChecker(ob.StateValidityCheckerFn( \
        partial(isStateValid, ss.getSpaceInformation())))

    si = ss.getSpaceInformation()
    planner = og.RRTstar(si)
    ss.setPlanner(planner)

    ss.getProblemDefinition().setOptimizationObjective(getThresholdPathLengthObj(si))
    
    return ss


def plan(planObj, initState, goalState):

    #space = ob.DubinsStateSpace(turningRadius=0.07)
    space = ob.SE2StateSpace()
    
    start = ob.State(space)
    start().setX(initState[0])
    start().setY(initState[1])
    start().setYaw(initState[2])

    goal = ob.State(space)
    goal().setX(goalState[0])
    goal().setY(goalState[1])
    goal().setYaw(goalState[2])
    
    planObj.setStartAndGoalStates(start, goal, 0.1)

    timeTerminationCondition = ob.timedPlannerTerminationCondition(4.0)
    exactSolTerminationCondition = ob.exactSolnPlannerTerminationCondition(planObj.getProblemDefinition())
    costTerminationCondition = ob.CostConvergenceTerminationCondition(planObj.getProblemDefinition(), epsilon=0.1)

    solved = planObj.solve(ob.plannerOrTerminationCondition(costTerminationCondition, timeTerminationCondition))

    if solved:
        try:
            planObj.simplifySolution()
        except:
            planObj.clear()

In [125]:
class PurePursuitController():

    k = 0.6  # look forward gain
    Lfc = 0.004  # look-ahead distance
    K_p = 1.2  #TODO speed control propotional gain
    K_i = 5.0  #TODO speed control integral gain
    K_d = 0.008  # speed control derivitive gain
    L = 0.324  # [m] wheel base of vehicle

    def __init__(self):
        self.traj_x = []
        self.traj_y = []
        self.target = None
        self.max_vel = 0.7
        # initialize with 0 velocity
        self.target_velocity = 0.0
        self.last_index = 0
        self.last_d = 100
        self.dt = 0.01
        self.vel_error_int_sum = 0
        self.vel_error = np.array([0])  
        self.vel_error_int = np.array([0]) 
        self.vel_error_der = np.array([0]) 
        self.is_finished = False

    def compute_control(self, state, target=None):
        steering = self.compute_steering(state, target)
        velocity = self.compute_velocity(state)
        return steering, velocity

    def compute_steering(self, state, target=None):
        if target is None:
            self.find_target(state)
        else:
            # allow manual setting of target
            self.target = target

        tx, ty = self.target
        alpha = math.atan2(ty - state.y, tx - state.x) - state.yaw
        if state.v < 0:  # back
            alpha = math.pi - alpha
        Lf = self.k * state.v + self.Lfc
        delta = math.atan2(2.0 * self.L * math.sin(alpha) / Lf, 1.0)
        return delta
                                   
    def compute_velocity(self, state):
        if self.is_finished:
            # stop moving if trajectory done
            return 0.0
        else:
            # speed control
            # Append the errors to the lists2
            self.vel_error     = np.append(self.vel_error      , self.target_velocity - state.v)
            self.vel_error_int = np.append(self.vel_error_int  , self.vel_error_int[-1] + self.vel_error[-1] * self.dt)
            self.vel_error_der = np.append(self.vel_error_der  , (self.vel_error[-1] - self.vel_error[-2]) / self.dt)

            P = self.vel_error[-1] * self.K_p
            I = self.vel_error_int[-1] * self.K_i
            D = self.vel_error_der[-1] * self.K_d

            correction = P + I + D

            # Saturating the velocity at a max velocity
            return self.target_velocity + min(correction, 0)

    def find_target(self, state):
        ind = self._calc_target_index(state)
        self.last_index = ind
        tx = self.traj_x[ind]
        ty = self.traj_y[ind]
        self.target = (tx, ty)

    def _calc_target_index(self, state):
        # search nearest point index
        dx = state.x - self.traj_x[self.last_index]
        dy = state.y - self.traj_y[self.last_index]
        d = math.sqrt(dx ** 2 + dy ** 2)
        
        ind = self.last_index

        if d < 0.05 or (d - self.last_d > 0 and ind == len(self.traj_x) - 1):
            ind += 1
            self.last_d = 100
        else:
            self.last_d = d

        if ind == len(self.traj_x) - 1:
            self.is_finished = True

        return ind
        

class VehicleState():
    def __init__(self):
        self.x = 0
        self.y = 0
        self.yaw = 0
        self.v = 0

In [ ]:
# generate planning scenario, plan, and perform path following

nData = 100000
nLatSamples = 3000 # number of samples to draw in latent space
p = 0.8 # likelihood which to sample from NN samples (when useNN is True)
pdef = problemDef(useNN=True)
maxPathLen = 40
maxCtrlIter = 600

dt = 0.4
L = 0.6
v = 0.1

trajSamples = np.zeros(shape=(1, 4)) # x, y, yaw, steer
initData = np.zeros(shape=(1, 3)) # x, y, yaw
goalData = np.zeros(shape=(1, 3))

plotPaths = False

nPlans = 0
planIdx = 0
while nPlans < nData and planIdx < numEntries - 1:

    while True:
        pdef.clear()

        c_sample_seed = cs[planIdx,:]
        c_sample = torch.from_numpy(np.repeat([c_sample_seed],nLatSamples,axis=0)).float().to(DEVICE)
        y = network(torch.randn(nLatSamples, z_dim).to(DEVICE), c_sample, encode=False).cpu().detach().numpy()

        start = data[planIdx, 4:8] # x, y, xdot, ydot
        goal = data[planIdx, 8:12]

        start_yaw = np.sign(int(start[-1] > 0) - 0.5) * np.arccos(np.dot(start[2:4] / np.linalg.norm(start[2:4]), [1, 0])) # get yaw from xdot, ydot
        goal_yaw = np.sign(int(goal[-1] > 0)- 0.5) * np.arccos(np.dot(goal[2:4] / np.linalg.norm(goal[2:4]), [1, 0]))

        plan(pdef, initState=np.append(start[:2], start_yaw), goalState=np.append(goal[:2], goal_yaw))

        planIdx += 1
        if pdef.getProblemDefinition().hasExactSolution():
            break

    sol = np.loadtxt(StringIO(pdef.getProblemDefinition().getSolutionPath().printAsMatrix()))

    controller = PurePursuitController()
    state = VehicleState()

    controller.traj_x = np.append(sol[:,0], goal[0])
    controller.traj_y = np.append(sol[:,1], goal[1])

    state.x = start[0]; state.y = start[1]; state.yaw = start_yaw; state.v = v
    statelist = [[state.x, state.y, state.yaw, 0]]

    ctrliter = 0
    while not controller.is_finished and ctrliter < maxCtrlIter :
        delta, _ = controller.compute_control(state)

        state.x += v * np.cos(state.yaw) * dt
        state.y += v * np.sin(state.yaw) * dt
        state.yaw += v / L * np.tan(delta) * dt

        statelist.append([state.x, state.y, state.yaw, delta])
        ctrliter += 1

    if ctrliter >= maxCtrlIter:
        continue

    statelist = statelist[1:] # remove first sample with 0 steering angle
    nSteps = len(statelist)
    statearr = np.array(statelist).reshape(-1).reshape(nSteps, 4)
    statearr[:, 3] = uniform_filter1d(statearr[:, 3], size=5, origin=2, mode='nearest', axis=0)

    dxdy = np.concatenate((sol[1:,0:2] - sol[:-1,0:2], (goal[:2] - sol[-1,0:2]).reshape(1, 2)), axis=0)
    xydot = dxdy / np.linalg.norm(dxdy, axis=1).reshape(sol.shape[0],1)

    if np.linalg.norm(sol[0, :2] - start[:2]) < 0.01:
        sol = sol[1:]
        xydot = xydot[1:]
    if np.linalg.norm(sol[-1, :2] - goal[:2]) < 0.01:
        sol = sol[:-1]
        xydot = xydot[:-1]

    nSamples = sol.shape[0]

    if nSamples > 0:

        trajSamples = np.vstack((trajSamples, statearr)) # x, y, xdot, ydot
        initData = np.vstack((initData, np.tile(np.append(start[0:2], start_yaw), reps=(nSteps, 1)))) # x, y, xdot, ydot
        goalData = np.vstack((goalData, np.tile(np.append(goal[0:2], goal_yaw), reps=(nSteps, 1)))) # x, y, xdot, ydot

        if plotPaths:
            fig1 = plt.figure(figsize=(10,6), dpi=80)
            ax1 = fig1.add_subplot(111, aspect='equal')
            ax1.scatter(y[:,0] * gridSize, y[:,1] * gridSize, color="green", s=70, alpha=0.1)
            ax1.quiver(sol[:, 0] * gridSize, sol[:, 1] * gridSize, xydot[:,0], xydot[:,1], color="purple", scale=8.0, width=0.015, alpha=0.9)
            ax1.quiver(start[0] * gridSize, start[1] * gridSize, start[2], start[3], color="red", scale=8.0, width=0.015) # init
            ax1.quiver(goal[0] * gridSize, goal[1] * gridSize, goal[2], goal[3], color="blue", scale=8.0, width=0.015) # goal
            ax1.scatter(statearr[:,0] * gridSize, statearr[:,1] * gridSize, color='purple', s=70)
            ax1.imshow(occGrid, extent=[0, gridSize, 0, gridSize], cmap='gray')
            plt.show()

            plt.plot(np.arange(nSteps), statearr[:, 3])

        nPlans += 1

In [121]:
print(planIdx)
steerdata = np.concatenate((trajSamples, initData, goalData), axis=1)[1:]
np.savetxt('/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_control/motion_planning/data/ctrlDataPfollow', steerdata, delimiter=',', fmt='%1.6f')

16
